<a href="https://colab.research.google.com/github/lageniaestela/12demayo/blob/master/notebook62db713860.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hola, esta es la reproduccion en español de un ejercicio realizado por Susan Li en el blog towardsdatascience (https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8)<br>
El post se publicó originalmente en 2017, y algunos de los algoritmos han tenido actualizaciones, pero creo que lo más significativo es que, pese a haber seguido los pasos, he obtenido un resultado mucho mejor (mayor accuracy, una matriz de confusion con menos predicciones incorrectas y mejores métricas) que el post original.<br>
Por favor, deja tus comentarios

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
#como hay varias variables que son float, vamos a ajustar a 2 el número de decimales que se muestran por defecto
pd.set_option('display.float_format', lambda x: '%.2f' % x)
print('número de decimales ajustado a dos decimales.')

La regresion logisitica es un algoritmo de clasificacion de Machine Learning que se usa para predecir la probabilidad de que una variable dependiente caiga en una u otra categoria, generalmente binaria Sí/No. <br>
Expresado en notación matemática, la regresion logistica te da la probabilidad P de que la variable Y, que es función de X, sea Sí <br>
P(Y=1)

# Requisitos de la Regresion logística.
* la variable dependiente debe ser binaria
* el valor 1 de la regresion debe representar el valor deseado.
* sólo se debe entrenar el modelo con variables independientes significativas.
* las variables independientes deben ser independientes entre si. Correlaciones bajas muestran poca colinealidad.
* las variables independientes deben ser directamente proporcionales a los ¿registros de probabilidad?
* Se necesitan muestras de tamaño considerable

# Data
El conjutno de datos viene del repositorio UCI Machine Repository, y contiene los datos de una campaña de marketing de un banco portugues. <br>
El objetivo de la clasificacion es predecir si el cliente se suscribirá a un producto financiero que se le ofrece. 1: si se suscribe, 0:no se suscribe.<br>
Para usar los datos, los he descargado en formato csv y los he subido a kaggle como un dataset. Ahora, en la derecha,le damos a Añadir data.

In [ ]:
# Importo los datos con una funcion de pandas que es read_csv.
# el archivo está dentro de input, y dentro de input, en banco.
# header = 0 es el valor por defecto, y significa que la primera fila,
# que es la fila 0, es la que contiene los encabecados.
data = pd.read_csv('/kaggle/input/banco/banco.csv', header = 0)


In [ ]:
#la funcion .shape nos informa del número de filas y de columnas
data_n_filas = data.shape[0]
data_n_columnas = data.shape[1]
print('El conjunto de datos tiene',data_n_columnas, 'columnas y',data_n_filas,'filas' )

In [ ]:
#vamos a ver cuantos valores perdidos hay
# aprovecho para sacar una copia de los datos, para evitar tampering.
data2=data.dropna()
data2_n_columnas = data.shape[0]
data2_n_filas = data.shape[1]
print('El conjunto de datos depurado tiene',data_n_columnas-data2_n_columnas, 'columnas menos y',data_n_filas -data2_n_filas,'filas' )

In [ ]:
# vamos a analizar las variables independientes:
print(list(data2.columns))

In [ ]:
def analiza(columna, dataset):
    #primero me va a informar de los datos nulos de la columna estudiada
    # isnull() nos dice si hay valores perdidos. Devuelve True o False
    # sum() nos da la suma de determinados valores.
    print('Esta columna tiene:',dataset[columna].isnull().sum(),'datos nulos')
    print('+++++')

    # luego me va a dar unos datos exploratorios.
    print(dataset[columna].describe())
    print('+++++')

    # y por ultimo, quiero que me diga las categorias
    #print(pd.value_counts(dataset[columna]))
    print(pd.unique(dataset[columna]))

In [ ]:
analiza('age',data2)

La columna age describe la edad de los encuestados, que oscila entre los 17 y los 98 años.

In [ ]:
analiza('job',data2)

job es una variable categorica con que separa los tipos de trabajos en 12 valores distintos.

In [ ]:
analiza('marital',data2)

marital es el estado civil. Clasifica a los encuestados en casados, solteros, divorciados y desconocido (Es curioso que no haya viudos, teniendo en cuenta que algunos encuestados eran casi centenarios)

In [ ]:
analiza('education',data2)

education hace referencia al nivel educativo.

In [ ]:
analiza('default',data2)

default es si el encuestado ha tenido algún aviso de impago.

In [ ]:
analiza('housing',data2)

housing indica si el encuestado tiene vivienda en propiedad

In [ ]:
analiza('loan',data2)

loan es si el encuestado está pagando algún tipo de crédito personal

In [ ]:
analiza('contact',data2)

contact es la forma de contacto

In [ ]:
analiza('month',data2)

month es en qué mes se le contacto por última vez

In [ ]:
analiza('day_of_week',data2)

day_of_week es en qué día de la semana se le contacto por última vez

In [ ]:
analiza('duration',data2)

Duracion de la llamada, en segundos. <br>
Este es un dato peculiar:
* si la llamada es dura 0 segundo, el resultado es No siempre.
* La duracion de la llamada no se puede saber antes de hacerla, de modo que no se puede usar como predictor de exito. Por tanto, no se debería incluir si lo que se pretende es tener un modelo predictivo realista.

In [ ]:
analiza('campaign',data2)

Campaign es el número de veces que se ha contactado a este cliente en esta campaña

In [ ]:
analiza('pdays',data2)

pdays son los dias pasados desde que el cliente fue contactado en una campaña anterior.

In [ ]:
analiza('previous',data2)

In [ ]:
previous es el número de veces que este cliente ha sido contactado en campañas anteriores

In [ ]:
analiza('poutcome',data2)

poutcome es el resultado de anteriores campañas de marketing

In [ ]:
analiza('emp_var_rate',data2)

tasa de variacion del empleo (?)

In [ ]:
analiza('cons_price_idx',data2)

indice de precios al consumidor (??)

In [ ]:
analiza('cons_conf_idx',data2)

indice de confianza del consumidor

In [ ]:
analiza('euribor3m',data2)

tasa a tres meses del euribor

In [ ]:
analiza('nr_employed',data2)

número de empleados (?) no entiendo. ¿Si el encuestado es un empresario y tiene empleados?

# Preparar las variables
## Education
La columna education tiene 8 categorias, y queremos reducirlas, de modo que las categorias basic.9y, basic.6y y basic.4y se van a quedar en solo basic.

In [ ]:
# usamos una funcion de NumPy que dice que en la columna education,
# donde el valor de la celda es basic.9y, hay que cambiarlo por Basic, en esa misma columna.
data2['education']=np.where(data2['education'] == 'basic.9y','Basic', data2['education'])
data2['education'].unique()

In [ ]:
# usamos una funcion de NumPy que dice que en la columna education,
# donde el valor de la celda es basic.9y, hay que cambiarlo por Basic, en esa misma columna.
data2['education']=np.where(data2['education'] == 'basic.6y','Basic', data2['education'])
data2['education'].unique()

In [ ]:
# usamos una funcion de NumPy que dice que en la columna education,
# donde el valor de la celda es basic.9y, hay que cambiarlo por Basic, en esa misma columna.
data2['education']=np.where(data2['education'] == 'basic.4y','Basic', data2['education'])
data2['education'].unique()

# Exploracion de dato

In [ ]:
# a ver cuanta gente se suscribió a ese producto
data2['y'].value_counts()

In [ ]:
# vamos a verlo en un gráfico
import matplotlib.pyplot as plt
plt.rc('font', size=14)

import seaborn as sns

# configuro colores
sns.set(style='white')
sns.set(style='whitegrid', color_codes=True)

In [ ]:
sns.countplot(x='y',
             data = data2,
             palette='hls')
plt.show
plt.savefig('count_plot')

Claramente, la muestra está desequilibrada: hay mucha más gente que no se suscribió, lo cual es un problema de cara a entrenar el modelo.

In [ ]:
# voy a sacar las medias de las variables cuantitativas según la columna y
data2.groupby('y').mean()

Observaciones. Entre ambos grupos:
* la edad no es una diferencia significativa.
* las llamadas exitosas fueron más largas.
* el número de veces que se contacto a los que dijeron que si fue inferior
* pasaron menos dias entre contactos entre los que dijeron que si. Se argumenta que a menos dias, mejor memoria de la llamada anterior y mejores posibilidades de una venta.

In [ ]:
# voy a sacar las modas de todas las variables según la columna y
data2.groupby('y').agg(pd.Series.mode)

Observaciones. Entre ambos grupos:
* la edad no es una diferencia significativa.
* es trabajo más frecuente es de tipo administrativo.
* el estado civil es casado
* los que dijeron que si eran universitarios, frente a los que dijeron que no, que tenían estudios básicos.

# Visualizaciones

In [ ]:
tabla_cruzada_job = pd.crosstab(data2.job,data2.y)

In [ ]:
# a ver, esto es nuevo.
# Matplotlib Inline command is a magic command that makes
#the plots generated by matplotlib show into the IPython shell
#that we are running and not in a separate output window.
%matplotlib inline

#esto tambien es interesante: pide una tabla cruzada,
#y le pone .plot detras, y entre parentesis, el tipo de plot
tabla_cruzada_job.plot(kind='bar')

plt.title('Purchase Frequency for Job Title')
plt.xlabel('Job')
plt.ylabel('Frequency of Purchase')
plt.savefig('purchase_fre_job')

In [ ]:
table=pd.crosstab(data.marital,data.y)
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title('Stacked Bar Chart of Marital Status vs Purchase')
plt.xlabel('Marital Status')
plt.ylabel('Proportion of Customers')
plt.savefig('mariral_vs_pur_stack')

In [ ]:
table=pd.crosstab(data.education,data.y)
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title('Stacked Bar Chart of Education vs Purchase')
plt.xlabel('Education')
plt.ylabel('Proportion of Customers')
plt.savefig('edu_vs_pur_stack')

In [ ]:
pd.crosstab(data.day_of_week,data.y).plot(kind='bar')
plt.title('Purchase Frequency for Day of Week')
plt.xlabel('Day of Week')
plt.ylabel('Frequency of Purchase')
plt.savefig('pur_dayofweek_bar')

In [ ]:
pd.crosstab(data.month,data.y).plot(kind='bar')
plt.title('Purchase Frequency for Month')
plt.xlabel('Month')
plt.ylabel('Frequency of Purchase')
plt.savefig('pur_fre_month_bar')

In [ ]:
data.age.hist()
plt.title('Histogram of Age')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.savefig('hist_age')

In [ ]:
pd.crosstab(data.poutcome,data.y).plot(kind='bar')
plt.title('Purchase Frequency for Poutcome')
plt.xlabel('Poutcome')
plt.ylabel('Frequency of Purchase')
plt.savefig('pur_fre_pout_bar')

# Variables Dummy

Aqui no entiendo muy bien que hace la autora. Parece que su objetivo es quedarse con un dataframe que contenga, por una parte, las columnas que son cuantitativas, y por otra, columnas dummie de las variables cualitativas, lo que la deja con un montón de columnas.<br>
Como entiendo lo que quiere hacer, pero no cómo lo hace, voy hacerlo de otra manera:

In [ ]:
data2.columns

In [ ]:
# creo un dataframe solo con las columnas de las que quiero sacar dummies
banco_dummies=data2.drop(['age', 'duration', 'campaign', 'pdays',
       'previous', 'emp_var_rate', 'cons_price_idx',
       'cons_conf_idx', 'euribor3m', 'nr_employed', 'y'],
                         axis  = 1,
                         inplace = False)
banco_dummies.columns

In [ ]:
#creo otro dataframe con las columnas de las que no saco dummies
banco_no_dummies=data2.drop(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'day_of_week', 'poutcome'],
                         axis  = 1,
                         inplace = False)
banco_no_dummies.columns

In [ ]:
banco_dummies2=pd.get_dummies(banco_dummies)
banco_dummies2.head(5)

In [ ]:
#uno el dataset con los dummies con el dataset de las columnas de las que no saque dummies
data_final=banco_no_dummies.join(banco_dummies2)
data_final.columns

# Over-sampling usando SMOTE
Confieso que no tengo ni idea de lo que es Over-sampling. Creo que es crear muestras falsas <br>
SMOTE son las siglas de Synthetic Minority Oversampling Technique. Lo que hace es:
1. crear muestras sinteticas de la clase de la que haya menos casos.
2. elegir al azar los vecinos para crear nuevas observaciones.

In [ ]:
# conjunto de datos sin la columna y
X = data_final.drop(['y'], axis  = 1, inplace = False)
X

In [ ]:
# solo la columna y
y = pd.DataFrame(data_final['y'])
y

In [ ]:
#importo la funcion SMOTE
from imblearn.over_sampling import SMOTE

#configuro la funcion SMOTE
os = SMOTE(random_state=0)

print('SMOTE configurado')

In [ ]:
# separo la muestra para comprobar la validez del modelo
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

#print('X_train:')
#print(X_train)
#print('++++++')
#print('X_test:')
#print(X_test)
#print('++++++')
#print('y_train:')
#print(y_train)
#print('++++++')
#print('y_test:')
#print(y_test)

In [ ]:
columns = X_train.columns
columns

In [ ]:
#entrenamos el modelo.
# en el post original esto parece estar mal: puso sample cuando es resample
# y da error pq hay valores perdidos
os_data_X,os_data_y=os.fit_resample(X_train, y_train)

#print('os_data_X:')
#print(os_data_X)
#print('++++++')
#print('os_data_y:')
#print(os_data_y)

Esta orden ha creado unos 10.000 filas nuevas

In [ ]:
len (os_data_y)-len(data_final)

In [ ]:
#lo guardamos en un dataframe
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['y'])

In [ ]:
# we can Check the numbers of our data
print("length of oversampled data is ",len(os_data_X))
print("Number of no subscription in oversampled data",len(os_data_y[os_data_y['y']==0]))
print("Number of subscription",len(os_data_y[os_data_y['y']==1]))
print("Proportion of no subscription data in oversampled data is ",len(os_data_y[os_data_y['y']==0])/len(os_data_X))
print("Proportion of subscription data in oversampled data is ",len(os_data_y[os_data_y['y']==1])/len(os_data_X))

Ahora tenemos una muestra perfectamente equilibrada. <br>
Puedes ver que se ha aumentado la muestra solo del conjunto de datos de entrenamiento. Esto es asi pq al no usar la muestra de test para crear observaciones sinteticas, ninguna informacion pasa al modelo de entrenamiento.

# Eliminacion Recurrente de Atributos (Recursive Feature Elimination)
La idea es construir una y otra vez un modelo, eligiendo el atributo que funciona mejor y descartando el que funciona peor.<br>
El objetivo es elegir los atributos considerando conjuntos de atributos cada vez mas pequeños.

In [ ]:
#Hacemos una lista con los nombres de las columnas
data_final_vars=data_final.columns.values.tolist()
data_final_vars

In [ ]:
#no se muy bien qué hace aqui.
# primero sobreescribe la variable y
y=['y']

# ahora crea una lista, en la que parece filtrar las columnas
# el elemento i, uno por cada elemento en la lista data_final_vars, pero solo si no está en la variable y
X=[i for i in data_final_vars if i not in y]
X

In [ ]:
# importa los algoritmos
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
print('importacion efectuada')

In [ ]:
estimator = LogisticRegression(max_iter=5000)
print('estimador establecido')

In [ ]:
#los parametros de RFE son:
# estimador, que es la regresion logistica
# el numero de atributos, que es 20
selector = RFE(estimator, n_features_to_select=20)
print('selector establecido')

In [ ]:
os_data_y

In [ ]:
os_data_y.values

In [ ]:
os_data_y.values.ravel()

In [ ]:
#entrenas el modelos
# no se porque añade .values y .ravel() a os_data_y
#   df.values desmonta el dataframe y devuelve los valores de las celdas como un unico y larguisimo vector
#   df.ravel() devuelve los datos subyacentes como un ndarray, que es un array de muchas dimensiones.
#   es como si lo convirtiera en una lista.
rfe = selector.fit(os_data_X, os_data_y.values.ravel())
atributos_seleccionados = rfe.support_.tolist()
print(atributos_seleccionados)
print(rfe.ranking_)
#he tenido que incrementar el número de iteraciones de la regresion,
#de los 100 por defecto, a 5000
#eso aumenta el tiempo de procesamiento.
# pero ya no sale el mensaje de error.

In [ ]:
#ahora necesito identificar los atributos seleccionados

#creo una lista vacia donde ir guardando cosas
atributos_seleccionados2=[]

#por cada numero dentro de un rango tan largo como la lista X
for i in range(len(X)):
    # si esa posicion en la lista atributos_selecionados es Verdad
    if atributos_seleccionados[i] == True:
        #entonces añade esa posicion de la lista X a la lista atributos_seleccionados2
        atributos_seleccionados2.append(X[i])

atributos_seleccionados2

El algoritmo de Eliminacion Recursiva nos ha ayudado a seleccionar los atributos más interesantes.

In [ ]:
X=os_data_X[atributos_seleccionados2]
y=os_data_y['y']

# Implementando el modelo

In [ ]:
import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

De esta tabla nos fijamos en los p-valores, que son todos menores de 0.05, excepto 4 variables, que tenemos que retirar.

In [ ]:
X2=X.drop(['marital_unknown','housing_no','housing_unknown','loan_unknown'],
                         axis  = 1,
                         inplace = False)

In [ ]:
logit_model=sm.Logit(y,X2)
result=logit_model.fit()
print(result.summary2())

## Ajuste del modelo de Regresion Logistica

In [ ]:
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.3, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

No ha funcionado, se queda en blanco, y no se porqué.

Prediciendo los resultados del test y calculando la precisión

In [ ]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

## Matriz de confusión

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

Este resultado se interpreta como que tenemos 7648+6467 predicciones correctas, frente a 1208+18 prediciones incorrectas.
# Estimar la precisión y otros.
* La precision se calcula a partir del número de verdaderos positivos (tp) y del número de falsos positivos (fp). Intuituvamente, es la habilidad del clasificador de no etiquetar una muestra como positiva si es negativa. La formula es numero de casos verdaderos positivos entre el total de casos.
* El Recall es la habilidad del clasificador en encontrar todas las muestras positivas. El calculo es similar, pero entre el numero de tp y falsos negativos.
* El F-beta score va de 1 (mejor valor) a 0 (peor valor). Es una media ponderada de precision y recall. El factor de ponderacion es beta aplicado al recall. No se para qué sirve.
* El soporte es el número de ocurrencias de cada caso en el conjunto de datos y_test

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

Interpretación: De todo el conjunto de pruebas, el 93% del depósito a plazo promocionado fue el depósito a plazo que les gustó a los clientes. De todo el conjunto de pruebas, el 93% de los depósitos a plazo preferidos del cliente fueron promovidos.

# ROC Curve

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

La curva de característica operativa del receptor (ROC) es otra herramienta común utilizada con clasificadores binarios. <br>
La línea punteada representa la curva ROC de un clasificador puramente aleatorio; Un buen clasificador se mantiene lo más lejos posible de esa línea (hacia la esquina superior izquierda).